#Kshitij Mahajan - SS & C Technologies Test
###A contextual Question-Answer (Q&A) bot
#### Technolgy stack:

1.   Mistral 7B model
2.   Chroma DB
3.   Instruct Embedding
4.   Langchain





In [1]:
#Install necessary libraries and packages
!pip -q install langchain tiktoken chromadb pypdf transformers InstructorEmbedding
!pip -q install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.1/496.1 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 114.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 122.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#Check the version of langchain
!pip show langchain

Name: langchain
Version: 0.0.333
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## Load Mistral-7B model




In [3]:
#Import AutoConfig class of Auto to check the configurations of the model
from transformers import AutoConfig
#Use from_pretrained method of Autoclass to load the config
config = AutoConfig.from_pretrained('mistralai/Mistral-7B-Instruct-v0.1')


In [4]:
#Print the loaded configurations
config

MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.1",
  "architectures": [
    "MistralForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.35.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [5]:
#Import the BitsAndBytesConfig library to simplify the process of model quantization

from transformers import BitsAndBytesConfig
import torch

#Use the 4-bit quantization to reduce the memory usage by approximately fourfold
#Change compute data type to torch.bfloat16 to improve the speed

bnb_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_compute_dtype=torch.bfloat16)

In [6]:
import torch
import transformers

# Import AutoModelForCasualLM as Mistral has CausalLm configuration class

from transformers import AutoTokenizer, AutoModelForCausalLM

#Setup tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

# Load pre-trained Mistral model for causal language modeling
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                              quantization_config=bnb_config,
                                              device_map='auto',
                                              low_cpu_mem_usage=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

#Choose the text-generation pipeline for Mistral model, passing in the model and tokenizer
#Choose hyperparameters of the model

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0.5,
    top_p=0.95,
    repetition_penalty=1.15,
    do_sample=True)

#Create an instance of the model using created pipeline
local_llm = HuggingFacePipeline(pipeline=pipe)

#### Check if the LLM is working

In [10]:
#Sample question to check if model and pipeline is working
print(local_llm('What is the capital of England?'))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



London


# RAG using LangChain multi-doc retriever with ChromaDB + Instruct Embeddings
I have stored multiple pdf documents in the folder.

- Multiple Files - PDFs
- ChromaDB
- Local LLM
- Instuctor Embeddings


## Setting up LangChain


In [11]:

import os
#Import Chroma vector store from langchain
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

#Import DirectoryLoader and PyPDFLoader to load pdf files from the given folder
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install sentence_transformers

#Instead of using OpenAIembeddings I have chosen open source InstructEmbedding model to calculate the embeddings of documents
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=8d06c264db3829b338bba3381928437b6073b9109a0b2a42d9b40f6cd465f0c6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


## Documents for QA answering
### I have 5 research papers stored in the google drive folder which I will be using for QA answeing

In [12]:
#Mount google drive
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


## Load multiple pdf documents and process them.





In [22]:

#Load pdf files from the directory
loader = DirectoryLoader('./drive/MyDrive/Paprs/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = []

#Implement error handing using try-except block to handle cases where pdfs are not found or not accessible
try:
    documents = loader.load()
except FileNotFoundError:
    print("PDF file not found. Check the path where file is stored")
except PermissionError:
    print("PDF file not accessible: ")
except Exception as e:
    print(f"Unexpected error occurred: {e}")

#This example shows that error handling is working. I have given the wrong path hence files are not found

PDF file not found. Check the path where file is stored


In [23]:
#Load pdf files from the directory
loader = DirectoryLoader('./drive/MyDrive/Papers/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = []

#Implement error handing using try-except block to handle cases where pdfs are not found or not accessible
try:
    documents = loader.load()
except FileNotFoundError:
    print("PDF file not found. Check the path where file is stored")
except PermissionError:
    print("PDF file not accessible: ")
except Exception as e:
    print(f"Unexpected error occurred: {e}")

#Here pdf files are loaded without raising any error

In [24]:
#Use text splitter to split up the long pieces of text into chunks
#Set the maximum size of chunks and the maximum overlap between chunks to maintain some continuity between chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

## HF Instructor Embeddings to calculate the embeddings of chunks

In [25]:
#Import opensource hkunlp/instructor-xl embedding model
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


## Create the vector database to store embeddings

In [26]:
# Embed chunks using embedding model and store the embeddings to chroma vector store
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Create an instance of embedding model
embedding = instructor_embeddings

## Create a chroma vector database and store the embeddings
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

## Create a retreiver

In [27]:
# Create a retreiver to retreive relevant chunks from vector database.
#Provide the number of relevant chunks you would like to retreive. Here I have chosen three chunks.
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

## Prompt engineering to handle out of context questions


In [28]:
#Create a prompt template to handle out of context questions
from langchain.prompts import PromptTemplate
prompt_template = """For the question below based on the provided context,if the context provides insufficient information and the question cannot be directly answered,reply "Based on the context I cannot answer.'

CONTEXT: {context}

Question: {question}

Strict Rules: Answer the question based on the context only.If the context provides insufficient information and the question cannot be directly answered,reply "Based on the context I cannot answer."

Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

## Create the chain to answer questions

In [29]:
#Create the chain to answer questions
#Provide the components we built so far to the chain
#Provide LLM to generate the text using context
#Provide retreiver to retreive context from vector database
#Choose to return context/Source documents

chain_type_kwargs = {"prompt": PROMPT}
qa_chain = RetrievalQA.from_chain_type(llm=local_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  chain_type_kwargs=chain_type_kwargs)

In [30]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

## Testing the chat flow

In [31]:
# Asking the question which is mentioned in the context
query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 FlashAttention is a new attention algorithm that computes exact attention with far fewer memory accesses by
avoiding reading and writing the attention matrix to and from HBM. It achieves this by restructuring the
attention computation to split the input into blocks and making several passes over input blocks, thus
incrementally performing the softmax reduction (tiling). Additionally, it stores the softmax normalization
factor from the forward pass to quickly recompute attention on-chip in the backward pass, which is faster than
the standard approach of reading the intermediate attention matrix from HBM.


Sources:
drive/MyDrive/Papers/Flash-attention.pdf
drive/MyDrive/Papers/Flash-attention.pdf
drive/MyDrive/Papers/Flash-attention.pdf


In [32]:
#Asking the question which is mentioned in the context
query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Based on the provided context, IO-aware means implementing or optimizing deep learning techniques or modules
for efficient use of input/output operations. It takes into account the specific requirements of different
types of memory hierarchies and aims to minimize data transfer and other IO-related costs.


Sources:
drive/MyDrive/Papers/Flash-attention.pdf
drive/MyDrive/Papers/Flash-attention.pdf
drive/MyDrive/Papers/Flash-attention.pdf


In [33]:
#Asking the question which is out of context
query = "What is the capital of India?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Based on the context I cannot answer.


Sources:
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/toolformer.pdf


In [34]:
##Asking the question which is mentioned in the context
query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Based on the context, Toolformer is a language model trained to decide which APIs to call, when to call them,
what arguments to pass, and how to best incorporate the results into future token prediction. It is designed
to learn these capabilities in a self-supervised way using simple API calls, rather than relying on large
amounts of human annotations or limiting tool use to specific settings.


Sources:
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/toolformer.pdf


In [35]:
#Asking the question which is mentioned in the context
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Based on the context, toolformer can use different tools such as search engines, calculators, and translation
systems via simple API calls.


Sources:
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/toolformer.pdf


In [36]:
#Asking the question which is out of context
query = "What is the capital of England?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Based on the context I cannot answer.


Sources:
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/toolformer.pdf


In [37]:
##Asking the question which is out of context
query = "What is Football?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Based on the context I cannot answer.


Sources:
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/toolformer.pdf


In [39]:
#Asking the question which is out of context
query = "Who is Ronaldo?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Based on the context I cannot answer.


Sources:
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/toolformer.pdf
drive/MyDrive/Papers/ReACT.pdf


In [40]:
#Our retreiver is using semantic similarity to find the context
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7cb93898bfd0>)

In [41]:
#Print the prompt template
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

For the question below based on the provided context,if the context provides insufficient information and the question cannot be directly answered,reply "Based on the context I cannot answer.'

CONTEXT: {context}

Question: {question}

Strict Rules: Answer the question based on the context only.If the context provides insufficient information and the question cannot be directly answered,reply "Based on the context I cannot answer."

Answer:
